# RedisTimeseriesManager Example: Sensor Data

In this example, we are going to maintain data of two imaginary sensors. Each sensor provides two measurements: `temperature` and `humidity`.

The data is collected with the resolution of one minute. Then we compress(downsample) the data to hourly and daily resolutions. To compress the data, we consider the average value of temperature and the maximum value of humidity in each time frame.

We also want to keep 1-minute sensor data for just one week, 1-hour data for one month and respectively 1-day data for a year.
In this Example, we use the classifier 1(c1) to identify the building where the sensor is located and the classifier 2(c2) for the sensor.

In [1]:
import time, datetime, random
from pytz import timezone

from redis_timeseries_manager import RedisTimeseriesManager

settings = {
    'host': 'localhost',
    'port': 6379,
    'db': 13,
    'password': None,
}

class SensorData(RedisTimeseriesManager):
    _name = 'sensors'
    _lines = ['temp', 'hum']
    _timeframes = {
        'raw': {'retention_secs': 60*60*24*7}, # retention 7 day
        '1h': {'retention_secs': 60*60*24*30, 'bucket_size_secs': 60*60}, # retention 1 month; timeframe 3600 secs
        '1d': {'retention_secs': 60*60*24*365, 'bucket_size_secs': 60*60*24}, # retention 1 year; timeframe 86400 secs
    }

    #compaction rules
    def _create_rule(self, c1:str, c2:str, line:str, timeframe_name:str, timeframe_specs:str, source_key:str, dest_key:str):
        if line == 'temp':
            aggregation_type = 'avg'
        elif line == 'hum':
            aggregation_type = 'max'
        bucket_size_secs = timeframe_specs['bucket_size_secs']
        self._set_rule(source_key, dest_key, aggregation_type, bucket_size_secs)
    
    @staticmethod
    def print_data(data):
        for ts, temp, hum in data:
            print(f"{datetime.datetime.fromtimestamp(ts, tz=timezone('UTC')):%Y-%m-%d %H:%M:%S}, temp: {round(temp, 2)}, hum(max): {round(hum, 2)}")

In [2]:
sd = SensorData(**settings)

In [3]:
# creating timeseries for sensor1
sd.create(
    c1='building1',
    c2='sensor1',
)

(True, 'Create building1:sensor1 success!')

In [4]:
# creating timeseries for sensor2
sd.create(
    c1='building1',
    c2='sensor2',
)

(True, 'Create building1:sensor2 success!')

In [5]:
# adding sample data for sensors
sensor1 = []
sensor2 = []
# generating random data from 2020-01-01 to 2020-03-01; 1-min resolution
for ts in range(1577836800, 1583020800, 60):
    sensor1.append([ts, random.randint(20, 30), random.randint(70, 90)])
    sensor2.append([ts, random.randint(-10, 0), random.randint(10, 20)])
# adding data
sd.insert(
    data=sensor1,
    c1='building1',
    c2='sensor1',
)
sd.insert(
    data=sensor2,
    c1='building1',
    c2='sensor2',
)

(True, 172800)

In [6]:
# reading sensor1 first 10 raw data, the data older than one week is removed
data = sd.read(
    c1='building1',
    c2='sensor1',
    timeframe='raw',
)
sd.print_data(data[1][:10])

2020-02-22 23:59:00, temp: 20.0, hum(max): 74.0
2020-02-23 00:00:00, temp: 22.0, hum(max): 90.0
2020-02-23 00:01:00, temp: 23.0, hum(max): 73.0
2020-02-23 00:02:00, temp: 20.0, hum(max): 83.0
2020-02-23 00:03:00, temp: 23.0, hum(max): 70.0
2020-02-23 00:04:00, temp: 21.0, hum(max): 80.0
2020-02-23 00:05:00, temp: 23.0, hum(max): 88.0
2020-02-23 00:06:00, temp: 25.0, hum(max): 79.0
2020-02-23 00:07:00, temp: 26.0, hum(max): 77.0
2020-02-23 00:08:00, temp: 23.0, hum(max): 88.0


In [7]:
# sensor 1 last 10 data, timeframe 1h
data = sd.read_last_n_records(
    c1='building1',
    c2='sensor1',
    timeframe='1h',
    minimum_timestamp=0,
    n=10,
)
sd.print_data(data[2])

2020-02-29 13:00:00, temp: 24.57, hum(max): 90.0
2020-02-29 14:00:00, temp: 24.95, hum(max): 89.0
2020-02-29 15:00:00, temp: 24.4, hum(max): 90.0
2020-02-29 16:00:00, temp: 24.88, hum(max): 89.0
2020-02-29 17:00:00, temp: 24.62, hum(max): 90.0
2020-02-29 18:00:00, temp: 25.38, hum(max): 90.0
2020-02-29 19:00:00, temp: 25.52, hum(max): 90.0
2020-02-29 20:00:00, temp: 24.25, hum(max): 90.0
2020-02-29 21:00:00, temp: 24.93, hum(max): 90.0
2020-02-29 22:00:00, temp: 25.08, hum(max): 90.0


In [8]:
# sensor 2 last 10 data, timeframe 1d
data = sd.read_last_n_records(
    c1='building1',
    c2='sensor2',
    timeframe='1d',
    minimum_timestamp=0,
    n=10,
)
sd.print_data(data[2])

2020-02-19 00:00:00, temp: -5.15, hum(max): 20.0
2020-02-20 00:00:00, temp: -5.05, hum(max): 20.0
2020-02-21 00:00:00, temp: -4.95, hum(max): 20.0
2020-02-22 00:00:00, temp: -4.99, hum(max): 20.0
2020-02-23 00:00:00, temp: -5.07, hum(max): 20.0
2020-02-24 00:00:00, temp: -5.02, hum(max): 20.0
2020-02-25 00:00:00, temp: -5.09, hum(max): 20.0
2020-02-26 00:00:00, temp: -5.1, hum(max): 20.0
2020-02-27 00:00:00, temp: -4.96, hum(max): 20.0
2020-02-28 00:00:00, temp: -5.1, hum(max): 20.0


## Other Commands
In the background, some keys are created in the redis db. To inspect the list of keys, run the following command:

In [9]:
sd.query_index(return_key_names=True)[1]

['sensors:building1:sensor1:1d:hum',
 'sensors:building1:sensor1:1d:temp',
 'sensors:building1:sensor1:1h:hum',
 'sensors:building1:sensor1:1h:temp',
 'sensors:building1:sensor1:raw:hum',
 'sensors:building1:sensor1:raw:temp',
 'sensors:building1:sensor2:1d:hum',
 'sensors:building1:sensor2:1d:temp',
 'sensors:building1:sensor2:1h:hum',
 'sensors:building1:sensor2:1h:temp',
 'sensors:building1:sensor2:raw:hum',
 'sensors:building1:sensor2:raw:temp']

You can inspect the info of each key by running the `stats` command as shown in the next cell.

> Note that each key have lablels properly filled with respective data(`c1`, `c2`, `...`), so you can take advantage of *redis multi-timeseries commands* like [TS.MRANGE](https://redis.io/commands/ts.mrange/)


In [10]:
sd.stats('building1', 'sensor1', 'raw', 'temp').__dict__

{'rules': [[b'sensors:building1:sensor1:1h:temp', 3600000, b'AVG'],
  [b'sensors:building1:sensor1:1d:temp', 86400000, b'AVG']],
 'source_key': None,
 'chunk_count': 4,
 'memory_usage': 16963,
 'total_samples': 10081,
 'labels': {'tl': 'sensors',
  'c1': 'building1',
  'c2': 'sensor1',
  'line': 'temp',
  'timeframe': 'raw'},
 'retention_msecs': 604800000,
 'lastTimeStamp': 1583020740000,
 'first_time_stamp': 1582415940000,
 'chunk_size': 4096,
 'duplicate_policy': 'last'}